# 0. Setup & Imports

In [ ]:
import xgboost
from sklearn import tree
from sklearn import ensemble
from sklearn import naive_bayes
from sklearn import svm
from sklearn import linear_model
from sklearn import neural_network
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os

import etmiss_utils

In [ ]:
# Don't use file locking. This means you can run jobs concurrently.
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

# 1. Load Data

- data were stored as .h5 files for this project. Gabriel (AJB student) may be able to provide some example data or an ntuple->h5 workflow.

In [ ]:
data = h5py.File("data/sample_00_fixed.h5", "r")

The .h5 file must contain PU-corrected calorimeter images (SK, CSSK, etc.) and a 'truth image' (e.g. truth_barcode). Check what fields you have available:

In [ ]:
data.keys() # what fields are available?

Take a peak:

In [ ]:
entry = 2
plt.imshow(np.transpose(data["SK"][entry]))

cb =  plt.colorbar()
cb.ax.set_ylabel("ET [GeV]")
plt.xlabel("$\eta$")
plt.ylabel("$\phi$")

plt.yticks([0, 31, 63], ["$-\pi$", "$0$", "$\pi$"])
plt.xticks([0, 24, 49], ["$-2.5$", "$0$", "$2.5$"])

# 2. Prepare Data

In this notebook, the MET from individual input channels (SK, CSSK,...) is calculated and fed into a non-convolutional classifier. Change num_entries.

In [ ]:
features = ["SK", "VorSK", "CSSK", "cluster"]

def extract_features(dataset):
    num_entries = 100#dataset["entries"][0]
    num_features = len(features)
    X = np.empty((num_entries, num_features))
    y = np.empty((num_entries))
    for i in range(num_entries):
        for j, feature in enumerate(features):
            tmp = etmiss_utils.get_etmiss((dataset[feature][i][:, np.newaxis]))
            X[i][j] = tmp
        y[i] = etmiss_utils.get_etmiss((dataset["truth_nm_barcode"][i][:, np.newaxis]))
        
    return X, y

In [ ]:
X, y = extract_features(data)

The PU algorithms correlate fairly well with the truth MET (good!)
The lower the MET threshold, the harder the classification task.

In [ ]:
plt.scatter(X[:,0], y)
plt.xlabel("SoftKiller MET [GeV]")
plt.ylabel("Truth MET [GeV]")

In [ ]:
# define a threshold of 50 GeV
Y = y>50
Y = Y.astype(np.int)

In [ ]:
# Observe the imbalance here. Not too good.
plt.hist(Y)
plt.xticks([0,1], ["Low MET", "High MET"])
plt.ylabel("Num Events")

## 2.1. Address Class Imbalance

In [ ]:
#1. Get class imbalance
length = len(Y)
num_high_met = sum(Y)

#2. Randomly choose some low-met events to get rid of
indices_low = np.argwhere(Y==0)[:,0]
indices_high = np.argwhere(Y==1)[:,0]
low_met_indices_to_keep = np.random.choice(indices_low, num_high_met)

#3. Fix the imbalance
Y = Y[np.concatenate([low_met_indices_to_keep, indices_high])]
X = X[np.concatenate([low_met_indices_to_keep, indices_high])]

In [ ]:
plt.hist(Y)
plt.xticks([0,1], ["Low MET", "High MET"])
plt.ylabel("Num Events")

In [ ]:
test_ratio = 0.05 #test on 20% of the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# 3. Set Up Pipeline

In [ ]:
algo_zoo = {
    'XGradientBoostTrees':xgboost.XGBClassifier,
    'DecisionTree': tree.DecisionTreeClassifier(criterion="entropy", max_depth=55),
    'AdaBoostTrees': ensemble.AdaBoostClassifier(n_estimators=85),
    'NaiveBayes':naive_bayes.GaussianNB(),
    'RFTrees':ensemble.RandomForestClassifier(n_estimators=270, max_depth=80, criterion="gini"),
    'LogisticRegressor':linear_model.LogisticRegression(),
    'MLP':neural_network.MLPClassifier(hidden_layer_sizes=(50,100,50), activation="tanh", solver="adam")
}

Some preprocessing tools that I used are demonstrated below (commented out right now). PowerTransformer does a box-cox transformation to make the data look gaussian (this is useful for the MLP).

In [ ]:
pca = PCA(n_components=2)
power = PowerTransformer(method="box-cox", standardize=True, copy=True)
scaler = StandardScaler()

#X_tr = power.fit_transform(X_train)
#X_tr = pca.fit_transform(X_train)
X_tr = X_train

#X_te = power.transform(X_test)
#X_te = pca.transform(X_test)
X_te = X_test

# 4. Train a model

Choose a model and train it

In [ ]:
clf = algo_zoo["MLP"]
clf.fit(X_tr, y_train)
predictions = clf.predict_proba(X_te)

# 5. Plot ROC

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, predictions[:,0])
plt.plot(tpr, fpr)
plt.xlabel("True +ve Rate")
plt.ylabel("False +ve Rate")
plt.legend([roc_auc_score(y_test, predictions[:,1])])

# 6. Try to understand models

How do the features (SK, VorSK, CSSK, cluster) contribute to the principal components?
The features must be standardised so the pca coefficients can be compared directly.

In [ ]:
#pca.components_

What proportion of the total variance does each component explain? The most useful component appears to be the mean MET (more or less).

In [ ]:
#pca.explained_variance_ratio_

What are the relative importances of the input features for this model?

In [ ]:
def get_feature_importances(clf):
    clf_type = type(clf).__name__.split(".")[-1]
    if clf_type == "MLPClassifier":
        # sum of the (absolute) input weights from each feature.
        # There are more sophisticated ways of assessing feature importance though.
        return np.sum(np.abs(clf.coefs_[0]), axis=1)
    if clf_type in {"DecisionTreeClassifier", "AdaBoostClassifier", "RandomForestClassifier", "XGBClassifier"}:
        # Uses a purity-based estimate of feature importance.
        return clf.feature_importances_
    if clf_type == "GaussianNB":
        return # you could do a permutation test here
    if clf_type == "LogisticRegression":
        # feature coeficients as used in the decision function.
        return clf.coef_

In [ ]:
get_feature_importances(clf)

# 7. Using GridSearchCV

In [ ]:
# choose a model to optimise
mdl = ensemble.AdaBoostClassifier()
# choose a grid of hyperparameters to search
param_grid = {"n_estimators":[50,100,150]}
# do the fit
clf = GridSearchCV(mdl, param_grid, verbose=1)
clf.fit(X,Y)

In [ ]:
print(clf.best_params_)